In [29]:

import pandas as pd
import numpy as np

from sklearn.metrics import (
    silhouette_score,
    calinski_harabasz_score,
    davies_bouldin_score,
    adjusted_rand_score,
    normalized_mutual_info_score
)

print("Librerías cargadas.")


Librerías cargadas.


In [30]:
# Dataset que contiene:
# - features originales
# - class (STAR/GALAXY/QSO)
# - cluster asignado por KMeans

df = pd.read_csv("../03_clustering/notebooks/kmeans_clusters.csv")

print(df.shape)
df.head()


(70000, 19)


,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,spec_obj_ID,redshift,plate,MJD,fiber_ID,class,cluster
0,1.237679e+18,357.361327,13.452999,19.29150,17.52694,16.51853,16.07618,15.73754,7773,301,6,293,6.918737e+18,0.122717,6145,56266,298,GALAXY,0
1,1.237679e+18,20.139775,12.406228,25.14513,21.52642,19.78986,18.74328,18.30575,7773,301,4,441,5.257002e+18,0.481416,4669,55831,638,GALAXY,2
2,1.237659e+18,212.955686,56.308421,17.24178,15.88844,15.42360,15.26871,15.22903,3225,301,2,51,2.755173e+18,-0.000120,2447,54498,350,STAR,0
3,1.237664e+18,6.001058,-0.817525,20.14468,18.25585,17.08372,16.59708,16.23867,4263,301,2,156,4.391194e+17,0.166363,390,51900,67,GALAXY,0
4,1.237661e+18,220.241593,40.908494,19.34401,18.96045,18.87436,18.89733,18.77630,3699,301,2,194,1.571771e+18,0.792190,1396,53112,53,QSO,2


In [31]:
# Asegurar columnas necesarias
assert "cluster" in df.columns, "Falta la columna cluster"
assert "class" in df.columns, "Falta la columna class"

clusters = df["cluster"]
true_labels = df["class"]

# Seleccionar variables numéricas originales (sin class, sin redshift, sin cluster)
X_numeric = df.select_dtypes(include=["float64", "int64"]).drop(
    columns=["redshift"], errors="ignore"
)


# Silhouette:

Mide qué tan bien separado está cada punto de los otros clusters.

1 → perfecto; 0 → en la frontera; <0 → mal asignado.

# Calinski-Harabasz:

Relación entre dispersión intra-cluster y separación inter-cluster.

Cuanto más alto, mejor.

### Intra-cluster:

Qué tan apretados/parecidos son los puntos dentro de cada grupo.

### Inter-cluster:

Qué tan separados están los grupos entre sí.

# Davies-Bouldin:

Mide cuán similares son los clusters entre sí.

0 es perfecto; cuanto más bajo, mejor.

Estas métricas solo usan la geometría de los datos, no saben nada de las clases reales.

In [32]:
# SILHOUETTE SCORE (0 a 1) – más alto es mejor
silhouette = silhouette_score(X_numeric, clusters)

# CALINSKI-HARABASZ – más alto es mejor
calinski = calinski_harabasz_score(X_numeric, clusters)

# DAVIES-BOULDIN – más bajo es mejor
davies = davies_bouldin_score(X_numeric, clusters)

print("=== Internal Metrics ===")
print(f"Silhouette Score:          {silhouette:.4f}")
print(f"Calinski-Harabasz Score:   {calinski:.4f}")
print(f"Davies-Bouldin Score:      {davies:.4f}")


=== Internal Metrics ===
Silhouette Score:          0.0858
Calinski-Harabasz Score:   21774.0401
Davies-Bouldin Score:      1.7273


# ARI:

Compara dos particiones (clases reales vs clusters).

1 significa que los clusters reproducen exactamente las clases.

0 es lo que esperarías de un clustering aleatorio.

# NMI:

Cuánta información comparten ambas particiones.

1 → perfecta correspondencia; 0 → independencia total.

In [33]:
# ADJUSTED RAND INDEX  (ARI)
# 1 = perfecto, 0 = aleatorio
ari = adjusted_rand_score(true_labels, clusters)

# NORMALIZED MUTUAL INFORMATION (NMI)
# 1 = perfecto
nmi = normalized_mutual_info_score(true_labels, clusters)

print("\n=== External Metrics ===")
print(f"ARI (Adjusted Rand Index): {ari:.4f}")
print(f"NMI (Mutual Information):  {nmi:.4f}")



=== External Metrics ===
ARI (Adjusted Rand Index): 0.0182
NMI (Mutual Information):  0.0461


In [34]:
print("\n=== Contingency Table (Cluster vs Class) ===\n")
cross = pd.crosstab(df["cluster"], df["class"])
print(cross)



=== Contingency Table (Cluster vs Class) ===

class    GALAXY   QSO  STAR
cluster                    
0         11642   606  6111
1         17649  7863  4109
2         12345  4851  4824


In [35]:
print("\n=== Descripción por Cluster ===\n")

cluster_desc = df.groupby("cluster").mean(numeric_only=True)
cluster_desc



=== Descripción por Cluster ===



,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,spec_obj_ID,redshift,plate,MJD,fiber_ID
cluster,,,,,,,,,,,,,,,,,
0,1.237663e+18,179.849655,24.020926,19.435543,17.855415,17.083487,16.726200,16.493161,4050.700474,301.0,3.473174,188.848358,2.787380e+18,0.108834,2475.604772,53851.894602,350.567624
1,1.237666e+18,177.368326,24.343684,23.601491,22.337701,21.295352,20.598920,20.223418,4747.634111,301.0,3.524358,179.060093,7.877123e+18,0.865245,6996.167651,56746.705277,502.048142
2,1.237665e+18,177.015195,24.064416,22.257912,20.658658,19.567005,19.019861,18.716868,4464.716349,301.0,3.522843,192.216712,5.457089e+18,0.582944,4846.755904,55472.694233,460.055540


In [36]:
print("\n=== VEREDICTO ===")

if silhouette > 0.5:
    print("Veredicto: VERDE — Clusters bien definidos.")
elif silhouette > 0.25:
    print("Veredicto: AMARILLO — Separación moderada, clusters parcialmente mezclados.")
else:
    print("Veredicto: ROJO — Clusters débiles, posible mezcla fuerte entre clases.")

print("\nInterpretación:")
print(f"- Buen Silhouette (>0.5): estructura geométrica clara.")
print(f"- Buen NMI/ARI (>0.6): clusters alineados a las clases reales.")
print(f"- Calinski alto y Davies bajo: geometría favorable.")



=== VEREDICTO ===
Veredicto: ROJO — Clusters débiles, posible mezcla fuerte entre clases.

Interpretación:
- Buen Silhouette (>0.5): estructura geométrica clara.
- Buen NMI/ARI (>0.6): clusters alineados a las clases reales.
- Calinski alto y Davies bajo: geometría favorable.
